In [1]:
import pandas as pd
import numpy as np


In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
train_purchases = pd.read_csv("datasets/train_purchases.csv")
train_sessions = pd.read_csv("datasets/train_sessions.csv")
item_features = pd.read_csv("datasets/item_features.csv")
candidate_items = pd.read_csv("datasets/candidate_items.csv")

In [4]:
test_leaderboard = pd.read_csv("datasets/test_leaderboard_sessions.csv")

In [5]:
test_leaderboard.head()

,session_id,item_id,date
0,26,19185,2021-06-16 09:53:54.158
1,200,17089,2021-06-25 12:23:40.811
2,200,17089,2021-06-25 12:24:36.631
3,200,8060,2021-06-25 12:24:41.677
4,200,4758,2021-06-25 12:24:50.692


In [6]:
train_purchases.head()

,session_id,item_id,date
0,3,15085,2020-12-18 21:26:47.986
1,13,18626,2020-03-13 19:36:15.507
2,18,24911,2020-08-26 19:20:32.049
3,19,12534,2020-11-02 17:16:45.92
4,24,13226,2020-02-26 18:27:44.114


In [7]:
train_sessions.head()

,session_id,item_id,date
0,3,9655,2020-12-18 21:25:00.373
1,3,9655,2020-12-18 21:19:48.093
2,13,15654,2020-03-13 19:35:27.136
3,18,18316,2020-08-26 19:18:30.833
4,18,2507,2020-08-26 19:16:31.211


In [8]:
print("Category number : ",len(item_features["feature_category_id"].unique()))

Category number :  73


In [9]:
item_features.head()

,item_id,feature_category_id,feature_value_id
0,2,56,365
1,2,62,801
2,2,68,351
3,2,33,802
4,2,72,75


In [10]:
candidate_items.head()

,item_id
0,4
1,8
2,9
3,19
4,20


In [11]:
train_purchases[train_purchases["item_id"] == 15085]

,session_id,item_id,date
0,3,15085,2020-12-18 21:26:47.986
81109,362500,15085,2020-11-20 15:43:18.18
156247,695904,15085,2020-12-11 08:57:55.779
208983,931085,15085,2020-12-09 14:30:26.118
295572,1315209,15085,2020-12-17 21:23:08.308
302413,1344939,15085,2021-01-18 18:14:13.989
310688,1381862,15085,2020-12-11 22:39:54.123
350089,1556254,15085,2020-12-06 21:38:37.59
363762,1616554,15085,2020-11-29 12:24:28.595
395474,1757080,15085,2020-12-11 12:46:46.343


In [12]:
fulldf = train_purchases.copy()

In [13]:
def get_feature_string(history):
    res = ""
    for k,itm in enumerate(history):
        item = (item_features[item_features["item_id"] == itm]["feature_category_id"].astype("str") 
                + item_features[item_features["item_id"] == itm]["feature_value_id"].astype("str")).astype("str").str.cat(sep=" ")

        if k == 0:
            res += item
        else:
            res += " " + item
    return res
def get_features():
    return fulldf["session_id"].apply(lambda x: get_feature_string(train_sessions[train_sessions["session_id"] == x]["item_id"]))

In [14]:
fulldf = fulldf[:100000]
test = item_features
test["test"] = test["feature_category_id"].astype("str") + test["feature_value_id"].astype("str")
lol = test.groupby("item_id")["test"].apply(' '.join).reset_index()

item_list = lol["item_id"]
fulldf["features"] = fulldf["item_id"].replace(lol["item_id"].values, lol["test"].values)

In [15]:
fulldf

,session_id,item_id,date,features
0,3,15085,2020-12-18 21:26:47.986,56365 47512 3745 49465 31768 51597 67349 6897 ...
1,13,18626,2020-03-13 19:36:15.507,56365 30715 69805 25376 17378 61462 32902 5762...
2,18,24911,2020-08-26 19:20:32.049,30847 59180 55267 6831 3793 26268 7275 5605 19...
3,19,12534,2020-11-02 17:16:45.92,69709 45559 68659 5605 19765 58299 3793 26268 ...
4,24,13226,2020-02-26 18:27:44.114,61462 29386 56365 28437 24588 4618 7275 3336 5...
...,...,...,...,...
99995,446492,11484,2021-01-09 20:13:05.855,65521 32902 47549 46825 56153 50240 18158 6386...
99996,446498,25815,2021-05-23 10:55:33.707,56365 47549 59856 17378 32286 68445 63861 7536...
99997,446499,13721,2020-11-27 17:00:16.065,65521 32522 4736 17100 6831 15451 45559 41145 ...
99998,446507,9296,2020-06-02 12:45:58.409,59856 45559 7275 17378 47273 25687 46825 32902...


In [16]:
cm = CountVectorizer().fit_transform(fulldf["features"])

In [ ]:
cs = cosine_similarity(cm)
print(cs)

In [ ]:
def get_real_id_for_item(itm_id):
    return fulldf[fulldf["item_id"] == itm_id].values[0][0]

real_id = get_real_id_for_item(15085)

In [ ]:
scores = list(enumerate(cs[real_id]))

In [ ]:
sorted_scores = sorted(scores, key = lambda x: x[1], reverse=True)
sorted_scores = sorted_scores[1:]

In [ ]:
print("TOP 5 >> Recommended item_id are : ")
for rec in sorted_scores[0:5]:
    print(fulldf[fulldf.index == rec[0]]["item_id"].values[0])

In [ ]:
def gencsv():
    df = pd.DataFrame()
    for index, row in df.iterrows():
        real_id = get_real_id_for_item(row["item_id"])
        scores = list(enumerate(cs[real_id]))
        sorted_scores = sorted(scores, key = lambda x: x[1], reverse=True)
        sorted_scores = sorted_scores[1:]
        for rec in sorted_scores[0:100]:
            outcsv = outcsv.append({'session_id' : row["session_id"], "item_id" : fulldf[fulldf.index == rec[0]]["item_id"].values[0], "rank": rec + 1}, ignore_index=True)
    
    df.to_csv("leaderboard_out.csv")